# Practical Python in Practice for Purposes of Proficient Programming

These tips will be more practical than the title I promise... :D 


### We all know and love optional arguments

Optional arguments are a great way to extend functionality while not having to update calling code in a million places. Take this simple divide method.

In [24]:
import math
def simple_divide(numerator, denominator, as_int=False):
    result = numerator / denominator
    
    if as_int:
        result = math.floor(result)
    return result

print(simple_divide(4,4, True))
print("Nice!")

1
Nice!


And sometimes simple becomes a bit more complicated.

In [12]:
def complex_divide(numerator, denominator, precision=4, as_int=False, ignore_div_by_zero=False):
    try:
        result = numerator / denominator
    except ZeroDivisionError:
        if ignore_div_by_zero:
            return float('inf')
        else:
            raise
    
    if as_int:
        result = math.floor(result)
    result = round(result, precision)
    return result

complex_divide(10, 3, 9, False, ignore_div_by_zero=True)

3.3333333333333335

We see here now the caller has many options of calling the method, which can possibly make it less maintainable. (What happens if we switch the order of the keyword arguments?)

How can we help callers of our code keep their use somewhat readable so that we can continue to add flags?
We the solution most of us might know is to use Keyword-Only arguments!

In [25]:
def better_complex_divide(numerator, denominator, *, precision=4, as_int=False, ignore_div_by_zero=False):
    """ Note the '*' added in the arguments. This forces all following fields to be passed by keyword"""
    try:
        result = numerator / denominator
    except ZeroDivisionError:
        if ignore_div_by_zero:
            return float('inf')
        else:
            raise
    
    if as_int:
        result = math.floor(result)
    result = round(result, precision)
    return result

print(better_complex_divide(10, 3, precision=4, ignore_div_by_zero=True))

3.3333


We can even take this a step further

Introduced in Python 3.8: Positional-only arguments

In [26]:
def best_complex_divide(numerator, denominator, /, precision=4, *, as_int=False, ignore_div_by_zero=False):
    try:
        result = numerator / denominator
    except ZeroDivisionError:
        if ignore_div_by_zero:
            return float('inf')
        else:
            raise
    
    if as_int:
        result = math.floor(result)
    result = round(result, precision)
    return result

print(best_complex_divide(10, 3, precision=4, ignore_div_by_zero=True))
print(best_complex_divide(10, 3, 4, as_int=True))

3.3333
3


### Practicality is the key to Comprehension

First off I want to start by echoing Item 27: "Use Comprehensions instead of map and filter"

In [31]:
my_list = [1,2,3,4,5,6]

doubled_evens_list_with_map = list(map(lambda x: x*2, filter(lambda x: x % 2 == 0, my_list))) # Yucky!! :/
doubled_evens_list_with_cmp = [item * 2 for item in my_list if item % 2 == 0] # Yummy!

print(doubled_evens_list_with_map)
print(doubled_evens_list_with_cmp)

[4, 8, 12]
[4, 8, 12]


However!!
Don't use more than two control expressions in a comprehension, just break it into a regular for loop. No one is going to enjoy trying to parse a large comprhension.

Something even more interesting that comes with Python 3.8 is when you mix assignment expressions (also known as: the walrus operator `:=`) you can leak variables from your comphrension.

In [38]:
stock = {"nails": 125, "screws": 35, "wingnuts": 8, "washers": 24}
order = ["screws", "wingnuts", "clips"]
def get_batches(count, size):
    return count // size

redundant_found = {name: get_batches(stock.get(name, 0), 8) for name in order if get_batches(stock.get(name, 0), 8)}
print(redundant_found)

efficient_found = {name: batches for name in order if (batches := get_batches(stock.get(name, 0), 8))}
print(efficient_found)

{'screws': 4, 'wingnuts': 1}
{'screws': 4, 'wingnuts': 1}


### Even generators can be Comprehendible!

Condsider generator comprehension expressions for large lazy comprehensions.

We all know generators are really useful for being memory efficient ways to iterate items, they even have their own comprehension syntax!

In [39]:
it = (len(x) for x in open("my_large_file.txt"))
it

<generator object <genexpr> at 0x7fb1b8bd79e0>

Generators are cool and memory efficient
But be defensive when working with them

### However, Buyer Beware

Generators themselves can cause a little trouble if not careful.
Take the following example of a perfectly normal function one may see in their day to day...

In [44]:
def normalize(iterable):
    total = sum(iterable)
    result = []
    for v in iterable:
        percent = v / total
        result.append(percent)
    return result

def my_iter(total):
    for i in range(total):
        yield i
    
normalize(my_iter(10))

[]

Why doesn't this work? 
Well, generators get "exhausted" once looped over.
E.g `sum(iterable)` exhausted the iterable then the `for` loop had nothing remaining to iterate over.

In [45]:
iterable = my_iter(10)
print(list(iterable))
print(list(iterable))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[]


### Be defensive when creating generators

By creating a class that defines the `__iter__` method!

In [51]:
class MyIterable:
    def __init__(self, total):
        self.total = total
    def __iter__(self):
        for i in range(self.total):
            yield i

print(f"It works now!! {normalize(MyIterable(10))}")

iterable = MyIterable(10)
print(list(iterable))
print(list(iterable))

It works now!! [0.0, 2.2222222222222223, 4.444444444444445, 6.666666666666667, 8.88888888888889, 11.11111111111111, 13.333333333333334, 15.555555555555555, 17.77777777777778, 20.0]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


### Let's make like a DJ and Mix it up a bit

The concept of Mixins is not Specific to Python, but it is achievable.
Mixins allow us to add functionality to Python classes without interfering too much with multiple inheritance.

To make a proper Mixin, the mixin must not require any touching of the `__init__` method. See the example below for a simple mixin.

In [61]:
import sys
class AttrCounterMixin:
    def attr_count(self):
        if isinstance(self, dict) or isinstance(self, tuple):
            return len(self)
        else:
            return len(vars(self))

class DataObject(AttrCounterMixin):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.x_y = x + y

do = DataObject(2, 3)
do.attr_count()

3

### Inherit from collections.abc for custom containers

https://docs.python.org/3/library/collections.abc.html#collections-abstract-base-classes